In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score , average_precision_score 
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve ,auc , log_loss ,  classification_report 
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import numpy as np

# Data Processing scenario 1
- fill null data and convert str to int
- 64%(0.8 * 0.8) data to train
- 16%(0.8 * 0.2) to validate
- 20% to test
- f1 : 0.35

In [42]:
import pandas as pd
with open('/content/drive/My Drive/fintech-introduction/term-project/train_sorted_strConverted_fillNA.csv', 'r') as f:
  df = pd.read_csv(f)
  # df = df.set_index('txkey')


# print(df_X)
# print(df_y)
df

,txkey,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,fraud_ind,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd
0,1547337,6767,137279,176279,785.96,5,62,0,5,0,0,0,5,0,0,1,70602.0,191,16615,0,5817,102,0
1,361558,6032,95126,90258,325.94,5,62,0,4,0,0,0,5,0,0,1,192920.0,251,78641,0,5817,102,0
2,1833662,6215,104992,168046,288.05,5,62,1,2,0,0,0,5,0,0,1,160124.0,380,54238,0,6580,38,0
3,1119486,6717,158233,33494,796.90,5,62,0,5,0,0,0,5,0,0,1,153353.0,304,33656,0,5800,102,0
4,1047761,5975,78764,162106,800.02,5,62,0,4,0,0,0,5,0,0,1,191448.0,263,93792,0,5817,102,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,1927172,6618,65727,181927,228.54,5,62,1,2,0,0,0,5,0,0,90,165427.0,192,65472,0,6593,75,0
1521783,239570,6769,43220,18630,939.74,5,62,1,8,0,1,0,5,0,0,90,124232.0,373,79200,0,5817,102,0
1521784,1459534,6767,18611,35515,383.76,5,62,0,5,0,0,0,5,0,0,90,140137.0,247,6484,0,5817,102,0
1521785,580748,6881,7613,38076,513.80,5,0,0,0,0,0,0,5,0,0,90,180243.0,457,59333,0,0,102,0


In [43]:
df_y = df['fraud_ind']
df_x = df.drop(['fraud_ind', 'txkey'], 1)
df_x

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd
0,6767,137279,176279,785.96,5,62,0,5,0,0,5,0,0,1,70602.0,191,16615,0,5817,102,0
1,6032,95126,90258,325.94,5,62,0,4,0,0,5,0,0,1,192920.0,251,78641,0,5817,102,0
2,6215,104992,168046,288.05,5,62,1,2,0,0,5,0,0,1,160124.0,380,54238,0,6580,38,0
3,6717,158233,33494,796.90,5,62,0,5,0,0,5,0,0,1,153353.0,304,33656,0,5800,102,0
4,5975,78764,162106,800.02,5,62,0,4,0,0,5,0,0,1,191448.0,263,93792,0,5817,102,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,6618,65727,181927,228.54,5,62,1,2,0,0,5,0,0,90,165427.0,192,65472,0,6593,75,0
1521783,6769,43220,18630,939.74,5,62,1,8,0,1,5,0,0,90,124232.0,373,79200,0,5817,102,0
1521784,6767,18611,35515,383.76,5,62,0,5,0,0,5,0,0,90,140137.0,247,6484,0,5817,102,0
1521785,6881,7613,38076,513.80,5,0,0,0,0,0,5,0,0,90,180243.0,457,59333,0,0,102,0


In [44]:
# split train+val and test
df_x_data, df_x_test = train_test_split(df_x, test_size=0.2, shuffle=False)
df_y_data = df_y.loc[df_x_data.index]
df_y_test = df_y.loc[df_x_test.index]

# print(df_x_data)
# print(df_x_test)
# print(df_y_data)
# print(df_y_test)

#split train and val
df_x_train, df_x_val = train_test_split(df_x_data, test_size=0.2, shuffle=False)
df_y_train = df_y_data.loc[df_x_train.index]
df_y_val = df_y_data.loc[df_x_val.index]


print('df_x_train:')
print(df_x_train)
print('df_x_val:')
print(df_x_val)
print('df_x_test:')
print(df_x_test)
print('df_y_train:')
print(df_y_train)
print('df_y_val:')
print(df_y_val)
print('df_y_test:')
print(df_y_test)

df_x_train:
        acqic   bacno    cano    conam  ...  ovrlt  scity  stocn  stscd
0        6767  137279  176279   785.96  ...      0   5817    102      0
1        6032   95126   90258   325.94  ...      0   5817    102      0
2        6215  104992  168046   288.05  ...      0   6580     38      0
3        6717  158233   33494   796.90  ...      0   5800    102      0
4        5975   78764  162106   800.02  ...      0   5817    102      0
...       ...     ...     ...      ...  ...    ...    ...    ...    ...
973938   6777   61498   91553  1154.25  ...      0   5817    102      0
973939   6625   87717   81548   366.38  ...      0   2260    102      0
973940   6769  110892  124186   352.55  ...      0   4907    102      0
973941   6215   60807   10755   168.20  ...      0   6580     38      0
973942   6769  121100  190558   834.05  ...      0   5817    102      0

[973943 rows x 21 columns]
df_x_val:
         acqic   bacno    cano    conam  ...  ovrlt  scity  stocn  stscd
973943    676

In [45]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_x_train, df_y_train)
lgb_eval = lgb.Dataset(df_x_val, df_y_val)
print(lgb_train.data)
print(lgb_eval.data)

        acqic   bacno    cano    conam  ...  ovrlt  scity  stocn  stscd
0        6767  137279  176279   785.96  ...      0   5817    102      0
1        6032   95126   90258   325.94  ...      0   5817    102      0
2        6215  104992  168046   288.05  ...      0   6580     38      0
3        6717  158233   33494   796.90  ...      0   5800    102      0
4        5975   78764  162106   800.02  ...      0   5817    102      0
...       ...     ...     ...      ...  ...    ...    ...    ...    ...
973938   6777   61498   91553  1154.25  ...      0   5817    102      0
973939   6625   87717   81548   366.38  ...      0   2260    102      0
973940   6769  110892  124186   352.55  ...      0   4907    102      0
973941   6215   60807   10755   168.20  ...      0   6580     38      0
973942   6769  121100  190558   834.05  ...      0   5817    102      0

[973943 rows x 21 columns]
         acqic   bacno    cano    conam  ...  ovrlt  scity  stocn  stscd
973943    6769   63692  126110   37

### Training

In [46]:
# specify your configurations as a dict

lgb_param = {
    'boosting_type' : 'gbdt',
    'objective' : 'cross_entropy',
    'metric' : 'cross_entropy',
    'learning_rate' : 0.1,
}


In [47]:
print('Starting training...')
max_round = 100
gbm = lgb.train(lgb_param,
         lgb_train,
         num_boost_round=max_round,
         valid_sets=lgb_eval,
         early_stopping_rounds=max_round)

Starting training...
[1]	valid_0's xentropy: 0.0460162
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's xentropy: 0.041752
[3]	valid_0's xentropy: 0.0399429
[4]	valid_0's xentropy: 0.0386895
[5]	valid_0's xentropy: 0.0377108
[6]	valid_0's xentropy: 0.0368728
[7]	valid_0's xentropy: 0.035784
[8]	valid_0's xentropy: 0.0350682
[9]	valid_0's xentropy: 0.0345539
[10]	valid_0's xentropy: 0.0340346
[11]	valid_0's xentropy: 0.0335469
[12]	valid_0's xentropy: 0.0330823
[13]	valid_0's xentropy: 0.032685
[14]	valid_0's xentropy: 0.0324141
[15]	valid_0's xentropy: 0.0321058
[16]	valid_0's xentropy: 0.0318205
[17]	valid_0's xentropy: 0.0315975
[18]	valid_0's xentropy: 0.0313886
[19]	valid_0's xentropy: 0.0311448
[20]	valid_0's xentropy: 0.0309428
[21]	valid_0's xentropy: 0.0326872
[22]	valid_0's xentropy: 0.0322847
[23]	valid_0's xentropy: 0.0321376
[24]	valid_0's xentropy: 0.0320096
[25]	valid_0's xentropy: 0.0319008
[26]	valid_0's xentropy: 0.0318044
[27]	valid_0's xen

### Show the performance on testing set


In [54]:
target_y = df_y_test
y_pred = gbm.predict(df_x_test, num_iteration=gbm.best_iteration)

# print(y_pred)
print('df_y_test.shape: {}'.format(df_y_test.shape))
print('auc: {}'.format(roc_auc_score(target_y, y_pred)))
th = 0.5

# row: actually, column: predict, the smaller class will be the target for counting the following indexies
cm = confusion_matrix(target_y, (y_pred > th).astype(int), labels=[0,1])
print(cm)
print('precision_score: {}'.format(precision_score(target_y, (y_pred > th).astype(int))))
print('recall_score: {}'.format(recall_score(target_y, (y_pred > th).astype(int))))
print('f1_score: {}'.format(f1_score(target_y, (y_pred > th).astype(int))))
print('accuracy: {}'.format(accuracy_score(target_y, (y_pred > th).astype(int))))



df_y_test.shape: (304358,)
auc: 0.9530781236009906
[[300819    710]
 [  2064    765]]
precision_score: 0.5186440677966102
recall_score: 0.2704135737009544
f1_score: 0.3554832713754647
accuracy: 0.9908857332483457


In [ ]:
# json_model = gbm.dump_model()
# import pprint
# pprint.pprint(json_model)

# Data Processing scenario 2
- Base on scenerio 1
- Balance two class
- Sample 40000 data in 64%(0.8 * 0.8) training data
- 16%(0.8 * 0.2) to validate
- 20% to test
- f1 : 0.16

In [ ]:
# Balance two #example in two class
# df_y_train
df_fraud = df_x_train.loc[df_y_train == 1]
df_non_fraud = df_x_train.loc[df_y_train == 0].sample(n=df_fraud.shape[0])
df_x_train_balance = pd.concat([df_fraud, df_non_fraud])
df_y_train_balance = pd.concat([df_y_train.loc[df_fraud.index], df_y_train.loc[df_non_fraud.index]])
df_fraud, df_non_fraud, df_x_train_balance, df_y_train_balance


(        acqic   bacno    cano    conam  ...  ovrlt  scity  stocn  stscd
 60       6215   16431   58243     0.00  ...      0   6580     38      0
 157      6760  147298  155705     0.00  ...      0   6593     75      0
 208      6210   37935   60807   785.90  ...      0    955     27      0
 224      3348  121994   20387   443.90  ...      0   3460     46      0
 274      3191  159517  146746     0.08  ...      0   5932     52      2
 ...       ...     ...     ...      ...  ...    ...    ...    ...    ...
 973222   5643    2671  117176   723.75  ...      1     15     38      0
 973509   6215  111181  207376   407.87  ...      0   3460     46      0
 973717   6760   98920  129272  2023.54  ...      0   2686     38      2
 973843   6760   37464  193146   556.79  ...      0   6593     75      0
 973924   6760   68389  160902   603.86  ...      0   6593     75      0
 
 [14789 rows x 21 columns],
         acqic   bacno    cano    conam  ...  ovrlt  scity  stocn  stscd
 820222   6769   2074

In [ ]:
df_x_train_balance

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd
60,6215,16431,58243,0.00,5,62,0,2,0,0,5,0,0,1,171329.0,380,54238,0,6580,38,0
157,6760,147298,155705,0.00,5,62,0,2,0,0,5,0,0,1,152100.0,192,66634,0,6593,75,0
208,6210,37935,60807,785.90,5,26,1,2,0,0,5,0,0,1,172318.0,286,57731,0,955,27,0
224,3348,121994,20387,443.90,5,62,1,8,0,0,5,0,0,1,215805.0,289,54828,0,3460,46,0
274,3191,159517,146746,0.08,4,26,1,8,0,0,5,0,0,1,212807.0,318,57084,0,5932,52,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776426,5975,133732,43953,407.87,5,62,0,5,0,0,5,0,0,46,173704.0,263,96273,0,5817,102,0
131131,6625,101506,106374,426.47,5,62,0,5,0,0,1,0,0,8,131054.0,291,78334,0,5817,102,0
551582,6032,13063,23587,611.19,5,62,1,2,0,0,5,0,0,33,201222.0,250,84286,0,5817,102,0
514549,3226,47505,186983,0.00,5,62,1,8,0,0,5,0,0,30,74338.0,192,90196,0,1458,102,0


In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_x_train_balance, df_y_train_balance)
lgb_eval = lgb.Dataset(df_x_val, df_y_val)
print(lgb_train.data)
print(lgb_eval.data)

        acqic   bacno    cano    conam  ...  ovrlt  scity  stocn  stscd
60       6215   16431   58243     0.00  ...      0   6580     38      0
157      6760  147298  155705     0.00  ...      0   6593     75      0
208      6210   37935   60807   785.90  ...      0    955     27      0
224      3348  121994   20387   443.90  ...      0   3460     46      0
274      3191  159517  146746     0.08  ...      0   5932     52      2
...       ...     ...     ...      ...  ...    ...    ...    ...    ...
93665    5975   57252  100308   674.61  ...      0   5817    102      0
756601      0  127983  181639   465.62  ...      0   5817    102      0
86353    3029  129811  122398   513.80  ...      0      0    102      0
961750   5975   81484  149662   657.53  ...      0   5817    102      0
958510   6716   58217  189351  1226.35  ...      0   5859    102      0

[29578 rows x 21 columns]
         acqic   bacno    cano    conam  ...  ovrlt  scity  stocn  stscd
973943    6769   63692  126110   370

### Training

In [ ]:
# specify your configurations as a dict

lgb_param = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'binary_logloss',
    'learning_rate' : 0.1,
}


In [ ]:
print('Starting training...')
max_round = 100
gbm = lgb.train(lgb_param,
         lgb_train,
         num_boost_round=max_round,
         valid_sets=lgb_eval,
         early_stopping_rounds=max_round)

Starting training...
[1]	valid_0's binary_logloss: 0.620853
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.561664
[3]	valid_0's binary_logloss: 0.513318
[4]	valid_0's binary_logloss: 0.472366
[5]	valid_0's binary_logloss: 0.437928
[6]	valid_0's binary_logloss: 0.408924
[7]	valid_0's binary_logloss: 0.383381
[8]	valid_0's binary_logloss: 0.362061
[9]	valid_0's binary_logloss: 0.344745
[10]	valid_0's binary_logloss: 0.328504
[11]	valid_0's binary_logloss: 0.315341
[12]	valid_0's binary_logloss: 0.304021
[13]	valid_0's binary_logloss: 0.293245
[14]	valid_0's binary_logloss: 0.283579
[15]	valid_0's binary_logloss: 0.27479
[16]	valid_0's binary_logloss: 0.267222
[17]	valid_0's binary_logloss: 0.260642
[18]	valid_0's binary_logloss: 0.25455
[19]	valid_0's binary_logloss: 0.24985
[20]	valid_0's binary_logloss: 0.245409
[21]	valid_0's binary_logloss: 0.241597
[22]	valid_0's binary_logloss: 0.23764
[23]	valid_0's binary_logloss: 0.233373
[24]	vali

### Show the performance on testing set


In [ ]:
target_y = df_y_test
y_pred = gbm.predict(df_x_test, num_iteration=gbm.best_iteration)

# print(y_pred)
print('df_y_test.shape: {}'.format(df_y_test.shape))
print('auc: {}'.format(roc_auc_score(target_y, y_pred)))
th = 0.5

# row: actually, column: predict, the smaller class will be the target for counting the following indexies
cm = confusion_matrix(target_y, (y_pred > th).astype(int), labels=[0,1])
print(cm)
print('precision_score: {}'.format(precision_score(target_y, (y_pred > th).astype(int))))
print('recall_score: {}'.format(recall_score(target_y, (y_pred > th).astype(int))))
print('f1_score: {}'.format(f1_score(target_y, (y_pred > th).astype(int))))
print('accuracy: {}'.format(accuracy_score(target_y, (y_pred > th).astype(int))))



df_y_test.shape: (304358,)
auc: 0.9566012238547966
[[276951  24578]
 [   362   2467]]
precision_score: 0.09121833980403032
recall_score: 0.872039589961117
f1_score: 0.16516034009506594
accuracy: 0.9180570249508803


# Data Processing scenario 3
- Base on scenerio 1
- Convert locdt and loctm
- 64%(0.8 * 0.8) training data
- 16%(0.8 * 0.2) to validate
- 20% to test
- f1 : 0.35

In [ ]:
import pandas as pd
with open('/content/drive/My Drive/fintech-introduction/term-project/train_locdt_loctm_converted.csv', 'r') as f:
  df = pd.read_csv(f)
  # df = df.set_index('txkey')


# print(df_X)
# print(df_y)
df

,txkey,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,fraud_ind,hcefg,insfg,iterm,locdt,mcc,mchno,ovrlt,scity,stocn,stscd,hh,mm,ss
0,1547337,6767,137279,176279,785.96,5,62,0,5,0,0,0,5,0,0,1,191,16615,0,5817,102,0,7,6,2
1,361558,6032,95126,90258,325.94,5,62,0,4,0,0,0,5,0,0,1,251,78641,0,5817,102,0,19,29,20
2,1833662,6215,104992,168046,288.05,5,62,1,2,0,0,0,5,0,0,1,380,54238,0,6580,38,0,16,1,24
3,1119486,6717,158233,33494,796.90,5,62,0,5,0,0,0,5,0,0,1,304,33656,0,5800,102,0,15,33,53
4,1047761,5975,78764,162106,800.02,5,62,0,4,0,0,0,5,0,0,1,263,93792,0,5817,102,0,19,14,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,1927172,6618,65727,181927,228.54,5,62,1,2,0,0,0,5,0,0,6,192,65472,0,6593,75,0,16,54,27
1521783,239570,6769,43220,18630,939.74,5,62,1,8,0,1,0,5,0,0,6,373,79200,0,5817,102,0,12,42,32
1521784,1459534,6767,18611,35515,383.76,5,62,0,5,0,0,0,5,0,0,6,247,6484,0,5817,102,0,14,1,37
1521785,580748,6881,7613,38076,513.80,5,0,0,0,0,0,0,5,0,0,6,457,59333,0,0,102,0,18,2,43


In [ ]:
df_y = df['fraud_ind']
df_x = df.drop(['fraud_ind', 'txkey'], 1)
df_x

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,mcc,mchno,ovrlt,scity,stocn,stscd,hh,mm,ss
0,6767,137279,176279,785.96,5,62,0,5,0,0,5,0,0,1,191,16615,0,5817,102,0,7,6,2
1,6032,95126,90258,325.94,5,62,0,4,0,0,5,0,0,1,251,78641,0,5817,102,0,19,29,20
2,6215,104992,168046,288.05,5,62,1,2,0,0,5,0,0,1,380,54238,0,6580,38,0,16,1,24
3,6717,158233,33494,796.90,5,62,0,5,0,0,5,0,0,1,304,33656,0,5800,102,0,15,33,53
4,5975,78764,162106,800.02,5,62,0,4,0,0,5,0,0,1,263,93792,0,5817,102,0,19,14,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,6618,65727,181927,228.54,5,62,1,2,0,0,5,0,0,6,192,65472,0,6593,75,0,16,54,27
1521783,6769,43220,18630,939.74,5,62,1,8,0,1,5,0,0,6,373,79200,0,5817,102,0,12,42,32
1521784,6767,18611,35515,383.76,5,62,0,5,0,0,5,0,0,6,247,6484,0,5817,102,0,14,1,37
1521785,6881,7613,38076,513.80,5,0,0,0,0,0,5,0,0,6,457,59333,0,0,102,0,18,2,43


In [ ]:
# split train+val and test
df_x_data, df_x_test = train_test_split(df_x, test_size=0.2, shuffle=False)
df_y_data = df_y.loc[df_x_data.index]
df_y_test = df_y.loc[df_x_test.index]

# print(df_x_data)
# print(df_x_test)
# print(df_y_data)
# print(df_y_test)

#split train and val
df_x_train, df_x_val = train_test_split(df_x_data, test_size=0.2, shuffle=False)
df_y_train = df_y_data.loc[df_x_train.index]
df_y_val = df_y_data.loc[df_x_val.index]


print('df_x_train:')
print(df_x_train)
print('df_x_val:')
print(df_x_val)
print('df_x_test:')
print(df_x_test)
print('df_y_train:')
print(df_y_train)
print('df_y_val:')
print(df_y_val)
print('df_y_test:')
print(df_y_test)

df_x_train:
        acqic   bacno    cano    conam  contp  ...  stocn  stscd  hh  mm  ss
0        6767  137279  176279   785.96      5  ...    102      0   7   6   2
1        6032   95126   90258   325.94      5  ...    102      0  19  29  20
2        6215  104992  168046   288.05      5  ...     38      0  16   1  24
3        6717  158233   33494   796.90      5  ...    102      0  15  33  53
4        5975   78764  162106   800.02      5  ...    102      0  19  14  48
...       ...     ...     ...      ...    ...  ...    ...    ...  ..  ..  ..
973938   6777   61498   91553  1154.25      5  ...    102      0  19  30  19
973939   6625   87717   81548   366.38      5  ...    102      0  16  36  56
973940   6769  110892  124186   352.55      5  ...    102      0  20  29   1
973941   6215   60807   10755   168.20      5  ...     38      0  16  29  58
973942   6769  121100  190558   834.05      5  ...    102      0  18  20  33

[973943 rows x 23 columns]
df_x_val:
         acqic   bacno    

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_x_train, df_y_train)
lgb_eval = lgb.Dataset(df_x_val, df_y_val)
print(lgb_train.data)
print(lgb_eval.data)

        acqic   bacno    cano    conam  contp  ...  stocn  stscd  hh  mm  ss
0        6767  137279  176279   785.96      5  ...    102      0   7   6   2
1        6032   95126   90258   325.94      5  ...    102      0  19  29  20
2        6215  104992  168046   288.05      5  ...     38      0  16   1  24
3        6717  158233   33494   796.90      5  ...    102      0  15  33  53
4        5975   78764  162106   800.02      5  ...    102      0  19  14  48
...       ...     ...     ...      ...    ...  ...    ...    ...  ..  ..  ..
973938   6777   61498   91553  1154.25      5  ...    102      0  19  30  19
973939   6625   87717   81548   366.38      5  ...    102      0  16  36  56
973940   6769  110892  124186   352.55      5  ...    102      0  20  29   1
973941   6215   60807   10755   168.20      5  ...     38      0  16  29  58
973942   6769  121100  190558   834.05      5  ...    102      0  18  20  33

[973943 rows x 23 columns]
         acqic   bacno    cano    conam  contp  

### Training

In [ ]:
# specify your configurations as a dict

lgb_param = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'binary_logloss',
    'learning_rate' : 0.1,
}


In [ ]:
print('Starting training...')
max_round = 100
gbm = lgb.train(lgb_param,
         lgb_train,
         num_boost_round=max_round,
         valid_sets=lgb_eval,
         early_stopping_rounds=max_round)

Starting training...
[1]	valid_0's binary_logloss: 0.0460265
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.042247
[3]	valid_0's binary_logloss: 0.040829
[4]	valid_0's binary_logloss: 0.0394245
[5]	valid_0's binary_logloss: 0.0383518
[6]	valid_0's binary_logloss: 0.0373206
[7]	valid_0's binary_logloss: 0.036428
[8]	valid_0's binary_logloss: 0.0357822
[9]	valid_0's binary_logloss: 0.0351848
[10]	valid_0's binary_logloss: 0.0344499
[11]	valid_0's binary_logloss: 0.0340252
[12]	valid_0's binary_logloss: 0.0335723
[13]	valid_0's binary_logloss: 0.033153
[14]	valid_0's binary_logloss: 0.0327879
[15]	valid_0's binary_logloss: 0.032476
[16]	valid_0's binary_logloss: 0.032187
[17]	valid_0's binary_logloss: 0.0319086
[18]	valid_0's binary_logloss: 0.031683
[19]	valid_0's binary_logloss: 0.031468
[20]	valid_0's binary_logloss: 0.0312923
[21]	valid_0's binary_logloss: 0.0311362
[22]	valid_0's binary_logloss: 0.031643
[23]	valid_0's binary_logloss: 0

### Show the performance on testing set

In [ ]:
target_y = df_y_test
y_pred = gbm.predict(df_x_test, num_iteration=gbm.best_iteration)

# print(y_pred)
print('df_y_test.shape: {}'.format(df_x_test.shape))
print('auc: {}'.format(roc_auc_score(target_y, y_pred)))
th = 0.5

# row: actually, column: predict, the smaller class will be the target for counting the following indexies
cm = confusion_matrix(target_y, (y_pred > th).astype(int), labels=[0,1])
print(cm)
print('precision_score: {}'.format(precision_score(target_y, (y_pred > th).astype(int))))
print('recall_score: {}'.format(recall_score(target_y, (y_pred > th).astype(int))))
print('f1_score: {}'.format(f1_score(target_y, (y_pred > th).astype(int))))
print('accuracy: {}'.format(accuracy_score(target_y, (y_pred > th).astype(int))))



df_y_test.shape: (304358, 23)
auc: 0.9577412735406008
[[300763    766]
 [  2060    769]]
precision_score: 0.5009771986970684
recall_score: 0.2718275008837045
f1_score: 0.3524289642529789
accuracy: 0.9907148818168079


# Data Processing scenario 4
- Base on scenerio 3
- Do normalization on numerical features
- Convert catogorical featrues to one hot style
- Only about 400000 data in total
- 64%(0.8 * 0.8) training data
- 16%(0.8 * 0.2) to validate
- 20% to test
- f1 : 0.32

In [ ]:
import pandas as pd
with open('/content/drive/My Drive/fintech-introduction/term-project/train_feature_normalization_onehot.csv', 'r') as f:
  df = pd.read_csv(f)
  # df = df.set_index('txkey')


# print(df_X)
# print(df_y)
df

,Unnamed: 0,acqic,bacno,cano,conam,mcc,mchno,scity,contp_0,contp_1,contp_2,contp_3,contp_4,contp_5,contp_6,csmcu_0,csmcu_1,csmcu_2,csmcu_3,csmcu_4,csmcu_5,csmcu_6,csmcu_7,csmcu_9,csmcu_10,csmcu_11,csmcu_12,csmcu_13,csmcu_14,csmcu_15,csmcu_16,csmcu_17,csmcu_18,csmcu_19,csmcu_20,csmcu_21,csmcu_22,csmcu_23,csmcu_24,csmcu_25,...,ss_21,ss_22,ss_23,ss_24,ss_25,ss_26,ss_27,ss_28,ss_29,ss_30,ss_31,ss_32,ss_33,ss_34,ss_35,ss_36,ss_37,ss_38,ss_39,ss_40,ss_41,ss_42,ss_43,ss_44,ss_45,ss_46,ss_47,ss_48,ss_49,ss_50,ss_51,ss_52,ss_53,ss_54,ss_55,ss_56,ss_57,ss_58,ss_59,fraud_ind
0,1,0.015972,0.275233,-0.306370,-0.816225,-0.600363,0.738111,0.536349,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,0.137776,0.483542,0.970861,-0.910290,1.054167,-0.053604,0.921739,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,5,0.505183,-0.269692,0.266075,0.998239,0.656567,-1.281195,0.536349,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,6,0.506515,1.151497,1.254999,-0.463005,-0.074504,0.753360,0.536349,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,7,0.148425,-0.272458,-0.339767,0.566122,-0.266892,-1.192852,0.536349,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456531,1521775,0.506515,-1.239088,0.879340,-0.566652,-0.600363,0.733667,-0.138461,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
456532,1521778,0.506515,1.251808,-0.941390,0.134700,-0.651666,0.734737,0.536349,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
456533,1521779,0.581061,1.019662,-1.460834,-0.349849,2.041755,0.111695,-2.401805,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
456534,1521781,-1.874977,-0.194189,1.676025,0.524985,0.258967,0.412445,0.594436,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:
df_y = df['fraud_ind']
df_x = df.drop(['fraud_ind', 'Unnamed: 0'], 1)
df_x

,acqic,bacno,cano,conam,mcc,mchno,scity,contp_0,contp_1,contp_2,contp_3,contp_4,contp_5,contp_6,csmcu_0,csmcu_1,csmcu_2,csmcu_3,csmcu_4,csmcu_5,csmcu_6,csmcu_7,csmcu_9,csmcu_10,csmcu_11,csmcu_12,csmcu_13,csmcu_14,csmcu_15,csmcu_16,csmcu_17,csmcu_18,csmcu_19,csmcu_20,csmcu_21,csmcu_22,csmcu_23,csmcu_24,csmcu_25,csmcu_26,...,ss_20,ss_21,ss_22,ss_23,ss_24,ss_25,ss_26,ss_27,ss_28,ss_29,ss_30,ss_31,ss_32,ss_33,ss_34,ss_35,ss_36,ss_37,ss_38,ss_39,ss_40,ss_41,ss_42,ss_43,ss_44,ss_45,ss_46,ss_47,ss_48,ss_49,ss_50,ss_51,ss_52,ss_53,ss_54,ss_55,ss_56,ss_57,ss_58,ss_59
0,0.015972,0.275233,-0.306370,-0.816225,-0.600363,0.738111,0.536349,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.137776,0.483542,0.970861,-0.910290,1.054167,-0.053604,0.921739,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.505183,-0.269692,0.266075,0.998239,0.656567,-1.281195,0.536349,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.506515,1.151497,1.254999,-0.463005,-0.074504,0.753360,0.536349,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.148425,-0.272458,-0.339767,0.566122,-0.266892,-1.192852,0.536349,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456531,0.506515,-1.239088,0.879340,-0.566652,-0.600363,0.733667,-0.138461,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456532,0.506515,1.251808,-0.941390,0.134700,-0.651666,0.734737,0.536349,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456533,0.581061,1.019662,-1.460834,-0.349849,2.041755,0.111695,-2.401805,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456534,-1.874977,-0.194189,1.676025,0.524985,0.258967,0.412445,0.594436,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:
# split train+val and test
df_x_data, df_x_test = train_test_split(df_x, test_size=0.2, shuffle=False)
df_y_data = df_y.loc[df_x_data.index]
df_y_test = df_y.loc[df_x_test.index]

# print(df_x_data)
# print(df_x_test)
# print(df_y_data)
# print(df_y_test)

#split train and val
df_x_train, df_x_val = train_test_split(df_x_data, test_size=0.2, shuffle=False)
df_y_train = df_y_data.loc[df_x_train.index]
df_y_val = df_y_data.loc[df_x_val.index]


print('df_x_train:')
print(df_x_train)
print('df_x_val:')
print(df_x_val)
print('df_x_test:')
print(df_x_test)
print('df_y_train:')
print(df_y_train)
print('df_y_val:')
print(df_y_val)
print('df_y_test:')
print(df_y_test)

df_x_train:
           acqic     bacno      cano     conam  ...  ss_56  ss_57  ss_58  ss_59
0       0.015972  0.275233 -0.306370 -0.816225  ...    0.0    0.0    0.0    0.0
1       0.137776  0.483542  0.970861 -0.910290  ...    0.0    0.0    0.0    0.0
2       0.505183 -0.269692  0.266075  0.998239  ...    0.0    0.0    0.0    0.0
3       0.506515  1.151497  1.254999 -0.463005  ...    0.0    0.0    0.0    0.0
4       0.148425 -0.272458 -0.339767  0.566122  ...    0.0    0.0    0.0    0.0
...          ...       ...       ...       ...  ...    ...    ...    ...    ...
292177  0.471238  0.803964 -0.201122 -0.217975  ...    0.0    0.0    0.0    0.0
292178 -0.021966 -1.333889 -0.129599  0.397182  ...    0.0    0.0    0.0    0.0
292179  0.015972 -1.581785 -1.297132 -0.719554  ...    0.0    0.0    0.0    0.0
292180  0.560428  1.225965  0.897187  0.087879  ...    0.0    0.0    0.0    0.0
292181 -3.998885  0.123024  1.271188 -0.469459  ...    0.0    0.0    0.0    0.0

[292182 rows x 386 columns]

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_x_train, df_y_train)
lgb_eval = lgb.Dataset(df_x_val, df_y_val)
print(lgb_train.data)
print(lgb_eval.data)

           acqic     bacno      cano     conam  ...  ss_56  ss_57  ss_58  ss_59
0       0.015972  0.275233 -0.306370 -0.816225  ...    0.0    0.0    0.0    0.0
1       0.137776  0.483542  0.970861 -0.910290  ...    0.0    0.0    0.0    0.0
2       0.505183 -0.269692  0.266075  0.998239  ...    0.0    0.0    0.0    0.0
3       0.506515  1.151497  1.254999 -0.463005  ...    0.0    0.0    0.0    0.0
4       0.148425 -0.272458 -0.339767  0.566122  ...    0.0    0.0    0.0    0.0
...          ...       ...       ...       ...  ...    ...    ...    ...    ...
292177  0.471238  0.803964 -0.201122 -0.217975  ...    0.0    0.0    0.0    0.0
292178 -0.021966 -1.333889 -0.129599  0.397182  ...    0.0    0.0    0.0    0.0
292179  0.015972 -1.581785 -1.297132 -0.719554  ...    0.0    0.0    0.0    0.0
292180  0.560428  1.225965  0.897187  0.087879  ...    0.0    0.0    0.0    0.0
292181 -3.998885  0.123024  1.271188 -0.469459  ...    0.0    0.0    0.0    0.0

[292182 rows x 386 columns]
           

### Training

In [ ]:
# specify your configurations as a dict

lgb_param = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'binary_logloss',
    'learning_rate' : 0.01,
}


In [ ]:
print('Starting training...')
max_round = 1000
gbm = lgb.train(lgb_param,
         lgb_train,
         num_boost_round=max_round,
         valid_sets=lgb_eval,
         early_stopping_rounds=max_round)

Starting training...
[1]	valid_0's binary_logloss: 0.0632379
Training until validation scores don't improve for 1000 rounds.
[2]	valid_0's binary_logloss: 0.0616314
[3]	valid_0's binary_logloss: 0.0603025
[4]	valid_0's binary_logloss: 0.0591934
[5]	valid_0's binary_logloss: 0.0582461
[6]	valid_0's binary_logloss: 0.0573858
[7]	valid_0's binary_logloss: 0.0566122
[8]	valid_0's binary_logloss: 0.0559183
[9]	valid_0's binary_logloss: 0.0552579
[10]	valid_0's binary_logloss: 0.0546494
[11]	valid_0's binary_logloss: 0.0540344
[12]	valid_0's binary_logloss: 0.0534822
[13]	valid_0's binary_logloss: 0.0529522
[14]	valid_0's binary_logloss: 0.0524484
[15]	valid_0's binary_logloss: 0.0519765
[16]	valid_0's binary_logloss: 0.051521
[17]	valid_0's binary_logloss: 0.0511029
[18]	valid_0's binary_logloss: 0.0507123
[19]	valid_0's binary_logloss: 0.0503395
[20]	valid_0's binary_logloss: 0.0499613
[21]	valid_0's binary_logloss: 0.0495867
[22]	valid_0's binary_logloss: 0.0492429
[23]	valid_0's binary_l

### Show the performance on testing set

In [ ]:
target_y = df_y_test
y_pred = gbm.predict(df_x_test, num_iteration=gbm.best_iteration)

# print(y_pred)
print('df_y_test.shape: {}'.format(df_x_test.shape))
print('auc: {}'.format(roc_auc_score(target_y, y_pred)))
th = 0.5

# row: actually, column: predict, the smaller class will be the target for counting the following indexies
cm = confusion_matrix(target_y, (y_pred > th).astype(int), labels=[0,1])
print(cm)
print('precision_score: {}'.format(precision_score(target_y, (y_pred > th).astype(int))))
print('recall_score: {}'.format(recall_score(target_y, (y_pred > th).astype(int))))
print('f1_score: {}'.format(f1_score(target_y, (y_pred > th).astype(int))))
print('accuracy: {}'.format(accuracy_score(target_y, (y_pred > th).astype(int))))



df_y_test.shape: (91308, 386)
auc: 0.955908048393172
[[90305   168]
 [  610   225]]
precision_score: 0.5725190839694656
recall_score: 0.2694610778443114
f1_score: 0.36644951140065146
accuracy: 0.9914793884435099


# Data Processing scenario 5
- Base on scenerio 3
- Do normalization on numerical features
- 64%(0.8 * 0.8) training data
- 16%(0.8 * 0.2) to validate
- 20% to test
- f1 : 0.34

In [55]:
import pandas as pd
with open('/content/drive/My Drive/fintech-introduction/term-project/train_feature_normalization.csv', 'r') as f:
  df = pd.read_csv(f)
  # df = df.set_index('txkey')


# print(df_X)
# print(df_y)
df

,Unnamed: 0,acqic,bacno,cano,conam,mcc,mchno,scity,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd,hh,mm,ss,fraud_ind
0,0,0.505183,1.165242,1.106042,0.325808,-1.369912,-1.274220,0.536349,5,62,0,5,0,0,5,0,0,1,0,102,0,7,6,2,0
1,1,0.015972,0.275233,-0.306370,-0.816225,-0.600363,0.738111,0.536349,5,62,0,4,0,0,5,0,0,1,0,102,0,19,29,20,0
2,2,0.137776,0.483542,0.970861,-0.910290,1.054167,-0.053604,0.921739,5,62,1,2,0,0,5,0,0,1,0,38,0,16,1,24,0
3,3,0.471904,1.607659,-1.238400,0.352968,0.079405,-0.721353,0.527763,5,62,0,5,0,0,5,0,0,1,0,102,0,15,33,53,0
4,4,-0.021966,-0.070230,0.873330,0.360713,-0.446453,1.229661,0.536349,5,62,0,4,0,0,5,0,0,1,0,102,0,19,14,48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,1521782,0.406010,-0.345490,1.198779,-1.058028,-1.357086,0.310865,0.928305,5,62,1,2,0,0,5,0,0,6,0,75,0,16,54,27,0
1521783,1521783,0.506515,-0.820698,-1.482458,0.707579,0.964387,0.756247,0.536349,5,62,1,8,0,1,5,0,0,6,0,102,0,12,42,32,0
1521784,1521784,0.505183,-1.340286,-1.205217,-0.672683,-0.651666,-1.602903,0.536349,5,62,0,5,0,0,5,0,0,6,0,102,0,14,1,37,0
1521785,1521785,0.581061,-1.572495,-1.163167,-0.349849,2.041755,0.111695,-2.401805,5,0,0,0,0,0,5,0,0,6,0,102,0,18,2,43,0


In [56]:
df_y = df['fraud_ind']
df_x = df.drop(['fraud_ind', 'Unnamed: 0'], 1)
df_x

,acqic,bacno,cano,conam,mcc,mchno,scity,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd,hh,mm,ss
0,0.505183,1.165242,1.106042,0.325808,-1.369912,-1.274220,0.536349,5,62,0,5,0,0,5,0,0,1,0,102,0,7,6,2
1,0.015972,0.275233,-0.306370,-0.816225,-0.600363,0.738111,0.536349,5,62,0,4,0,0,5,0,0,1,0,102,0,19,29,20
2,0.137776,0.483542,0.970861,-0.910290,1.054167,-0.053604,0.921739,5,62,1,2,0,0,5,0,0,1,0,38,0,16,1,24
3,0.471904,1.607659,-1.238400,0.352968,0.079405,-0.721353,0.527763,5,62,0,5,0,0,5,0,0,1,0,102,0,15,33,53
4,-0.021966,-0.070230,0.873330,0.360713,-0.446453,1.229661,0.536349,5,62,0,4,0,0,5,0,0,1,0,102,0,19,14,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,0.406010,-0.345490,1.198779,-1.058028,-1.357086,0.310865,0.928305,5,62,1,2,0,0,5,0,0,6,0,75,0,16,54,27
1521783,0.506515,-0.820698,-1.482458,0.707579,0.964387,0.756247,0.536349,5,62,1,8,0,1,5,0,0,6,0,102,0,12,42,32
1521784,0.505183,-1.340286,-1.205217,-0.672683,-0.651666,-1.602903,0.536349,5,62,0,5,0,0,5,0,0,6,0,102,0,14,1,37
1521785,0.581061,-1.572495,-1.163167,-0.349849,2.041755,0.111695,-2.401805,5,0,0,0,0,0,5,0,0,6,0,102,0,18,2,43


In [57]:
# split train+val and test
df_x_data, df_x_test = train_test_split(df_x, test_size=0.2, shuffle=False)
df_y_data = df_y.loc[df_x_data.index]
df_y_test = df_y.loc[df_x_test.index]

# print(df_x_data)
# print(df_x_test)
# print(df_y_data)
# print(df_y_test)

#split train and val
df_x_train, df_x_val = train_test_split(df_x_data, test_size=0.2, shuffle=False)
df_y_train = df_y_data.loc[df_x_train.index]
df_y_val = df_y_data.loc[df_x_val.index]


print('df_x_train:')
print(df_x_train)
print('df_x_val:')
print(df_x_val)
print('df_x_test:')
print(df_x_test)
print('df_y_train:')
print(df_y_train)
print('df_y_val:')
print(df_y_val)
print('df_y_test:')
print(df_y_test)

df_x_train:
           acqic     bacno      cano     conam  ...  stscd  hh  mm  ss
0       0.505183  1.165242  1.106042  0.325808  ...      0   7   6   2
1       0.015972  0.275233 -0.306370 -0.816225  ...      0  19  29  20
2       0.137776  0.483542  0.970861 -0.910290  ...      0  16   1  24
3       0.471904  1.607659 -1.238400  0.352968  ...      0  15  33  53
4      -0.021966 -0.070230  0.873330  0.360713  ...      0  19  14  48
...          ...       ...       ...       ...  ...    ...  ..  ..  ..
973938  0.511839 -0.434780 -0.285107  1.240116  ...      0  19  30  19
973939  0.410669  0.118801 -0.449383 -0.715830  ...      0  16  36  56
973940  0.506515  0.608113  0.250707 -0.750164  ...      0  20  29   1
973941  0.137776 -0.449370 -1.611761 -1.207827  ...      0  16  29  58
973942  0.506515  0.823642  1.340495  0.445195  ...      0  18  20  33

[973943 rows x 23 columns]
df_x_val:
            acqic     bacno      cano     conam  ...  stscd  hh  mm  ss
973943   0.506515 -0.38845

In [58]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_x_train, df_y_train)
lgb_eval = lgb.Dataset(df_x_val, df_y_val)
print(lgb_train.data)
print(lgb_eval.data)

           acqic     bacno      cano     conam  ...  stscd  hh  mm  ss
0       0.505183  1.165242  1.106042  0.325808  ...      0   7   6   2
1       0.015972  0.275233 -0.306370 -0.816225  ...      0  19  29  20
2       0.137776  0.483542  0.970861 -0.910290  ...      0  16   1  24
3       0.471904  1.607659 -1.238400  0.352968  ...      0  15  33  53
4      -0.021966 -0.070230  0.873330  0.360713  ...      0  19  14  48
...          ...       ...       ...       ...  ...    ...  ..  ..  ..
973938  0.511839 -0.434780 -0.285107  1.240116  ...      0  19  30  19
973939  0.410669  0.118801 -0.449383 -0.715830  ...      0  16  36  56
973940  0.506515  0.608113  0.250707 -0.750164  ...      0  20  29   1
973941  0.137776 -0.449370 -1.611761 -1.207827  ...      0  16  29  58
973942  0.506515  0.823642  1.340495  0.445195  ...      0  18  20  33

[973943 rows x 23 columns]
            acqic     bacno      cano     conam  ...  stscd  hh  mm  ss
973943   0.506515 -0.388457  0.282298 -0.704758 

### Training


In [59]:
# specify your configurations as a dict

lgb_param = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'binary_logloss',
    'learning_rate' : 0.1,
}


In [60]:
print('Starting training...')
max_round = 100
gbm = lgb.train(lgb_param,
         lgb_train,
         num_boost_round=max_round,
         valid_sets=lgb_eval,
         early_stopping_rounds=max_round)

Starting training...
[1]	valid_0's binary_logloss: 0.0460561
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.0428908
[3]	valid_0's binary_logloss: 0.0408416
[4]	valid_0's binary_logloss: 0.0395564
[5]	valid_0's binary_logloss: 0.0384729
[6]	valid_0's binary_logloss: 0.0376248
[7]	valid_0's binary_logloss: 0.0367153
[8]	valid_0's binary_logloss: 0.0357086
[9]	valid_0's binary_logloss: 0.0350448
[10]	valid_0's binary_logloss: 0.0344275
[11]	valid_0's binary_logloss: 0.0339028
[12]	valid_0's binary_logloss: 0.0334791
[13]	valid_0's binary_logloss: 0.0330645
[14]	valid_0's binary_logloss: 0.0327595
[15]	valid_0's binary_logloss: 0.0324623
[16]	valid_0's binary_logloss: 0.0321454
[17]	valid_0's binary_logloss: 0.0319342
[18]	valid_0's binary_logloss: 0.0315517
[19]	valid_0's binary_logloss: 0.0313128
[20]	valid_0's binary_logloss: 0.03115
[21]	valid_0's binary_logloss: 0.0310039
[22]	valid_0's binary_logloss: 0.0308267
[23]	valid_0's binary_log

### Show the performance on testing set

In [61]:
target_y = df_y_test
y_pred = gbm.predict(df_x_test, num_iteration=gbm.best_iteration)

# print(y_pred)
print('df_y_test.shape: {}'.format(df_x_test.shape))
print('auc: {}'.format(roc_auc_score(target_y, y_pred)))
th = 0.5

# row: actually, column: predict, the smaller class will be the target for counting the following indexies
cm = confusion_matrix(target_y, (y_pred > th).astype(int), labels=[0,1])
print(cm)
print('precision_score: {}'.format(precision_score(target_y, (y_pred > th).astype(int))))
print('recall_score: {}'.format(recall_score(target_y, (y_pred > th).astype(int))))
print('f1_score: {}'.format(f1_score(target_y, (y_pred > th).astype(int))))
print('accuracy: {}'.format(accuracy_score(target_y, (y_pred > th).astype(int))))



df_y_test.shape: (304358, 23)
auc: 0.959458192823326
[[300803    726]
 [  2086    743]]
precision_score: 0.505786249149081
recall_score: 0.26263697419582893
f1_score: 0.34574220567705904
accuracy: 0.990760880279145


# Data Processing scenario 6
- Base on scenerio 1
- Only use the feature that selected by subjective
  - drop column : acquic, bacno, cano, mchno
- 64%(0.8 * 0.8) training data
- 16%(0.8 * 0.2) to validate
- 20% to test
- f1 : 0.34

In [ ]:
import pandas as pd
with open('/content/drive/My Drive/fintech-introduction/term-project/train_sorted_strConverted_fillNA.csv', 'r') as f:
  df = pd.read_csv(f)
  # df = df.set_index('txkey')


# print(df_X)
# print(df_y)
df

,txkey,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,fraud_ind,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd
0,1547337,6767,137279,176279,785.96,5,62,0,5,0,0,0,5,0,0,1,70602.0,191,16615,0,5817,102,0
1,361558,6032,95126,90258,325.94,5,62,0,4,0,0,0,5,0,0,1,192920.0,251,78641,0,5817,102,0
2,1833662,6215,104992,168046,288.05,5,62,1,2,0,0,0,5,0,0,1,160124.0,380,54238,0,6580,38,0
3,1119486,6717,158233,33494,796.90,5,62,0,5,0,0,0,5,0,0,1,153353.0,304,33656,0,5800,102,0
4,1047761,5975,78764,162106,800.02,5,62,0,4,0,0,0,5,0,0,1,191448.0,263,93792,0,5817,102,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,1927172,6618,65727,181927,228.54,5,62,1,2,0,0,0,5,0,0,90,165427.0,192,65472,0,6593,75,0
1521783,239570,6769,43220,18630,939.74,5,62,1,8,0,1,0,5,0,0,90,124232.0,373,79200,0,5817,102,0
1521784,1459534,6767,18611,35515,383.76,5,62,0,5,0,0,0,5,0,0,90,140137.0,247,6484,0,5817,102,0
1521785,580748,6881,7613,38076,513.80,5,0,0,0,0,0,0,5,0,0,90,180243.0,457,59333,0,0,102,0


In [ ]:
df_y = df['fraud_ind']
df_x = df.drop(['fraud_ind', 'txkey', 'acqic', 'bacno', 'cano', 'mchno'], 1)
df_x

,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,ovrlt,scity,stocn,stscd
0,785.96,5,62,0,5,0,0,5,0,0,1,70602.0,191,0,5817,102,0
1,325.94,5,62,0,4,0,0,5,0,0,1,192920.0,251,0,5817,102,0
2,288.05,5,62,1,2,0,0,5,0,0,1,160124.0,380,0,6580,38,0
3,796.90,5,62,0,5,0,0,5,0,0,1,153353.0,304,0,5800,102,0
4,800.02,5,62,0,4,0,0,5,0,0,1,191448.0,263,0,5817,102,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,228.54,5,62,1,2,0,0,5,0,0,90,165427.0,192,0,6593,75,0
1521783,939.74,5,62,1,8,0,1,5,0,0,90,124232.0,373,0,5817,102,0
1521784,383.76,5,62,0,5,0,0,5,0,0,90,140137.0,247,0,5817,102,0
1521785,513.80,5,0,0,0,0,0,5,0,0,90,180243.0,457,0,0,102,0


In [ ]:
# split train+val and test
df_x_data, df_x_test = train_test_split(df_x, test_size=0.2, shuffle=False)
df_y_data = df_y.loc[df_x_data.index]
df_y_test = df_y.loc[df_x_test.index]

# print(df_x_data)
# print(df_x_test)
# print(df_y_data)
# print(df_y_test)

#split train and val
df_x_train, df_x_val = train_test_split(df_x_data, test_size=0.2, shuffle=False)
df_y_train = df_y_data.loc[df_x_train.index]
df_y_val = df_y_data.loc[df_x_val.index]


print('df_x_train:')
print(df_x_train)
print('df_x_val:')
print(df_x_val)
print('df_x_test:')
print(df_x_test)
print('df_y_train:')
print(df_y_train)
print('df_y_val:')
print(df_y_val)
print('df_y_test:')
print(df_y_test)

df_x_train:
          conam  contp  csmcu  ecfg  etymd  ...  mcc  ovrlt  scity  stocn  stscd
0        785.96      5     62     0      5  ...  191      0   5817    102      0
1        325.94      5     62     0      4  ...  251      0   5817    102      0
2        288.05      5     62     1      2  ...  380      0   6580     38      0
3        796.90      5     62     0      5  ...  304      0   5800    102      0
4        800.02      5     62     0      4  ...  263      0   5817    102      0
...         ...    ...    ...   ...    ...  ...  ...    ...    ...    ...    ...
973938  1154.25      5     62     0      5  ...  357      0   5817    102      0
973939   366.38      5     62     0      5  ...  291      0   2260    102      0
973940   352.55      5     62     0      5  ...  251      0   4907    102      0
973941   168.20      5     62     1      2  ...  380      0   6580     38      0
973942   834.05      5     62     1      8  ...  373      0   5817    102      0

[973943 rows x 

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_x_train, df_y_train)
lgb_eval = lgb.Dataset(df_x_val, df_y_val)
print(lgb_train.data)
print(lgb_eval.data)

          conam  contp  csmcu  ecfg  etymd  ...  mcc  ovrlt  scity  stocn  stscd
0        785.96      5     62     0      5  ...  191      0   5817    102      0
1        325.94      5     62     0      4  ...  251      0   5817    102      0
2        288.05      5     62     1      2  ...  380      0   6580     38      0
3        796.90      5     62     0      5  ...  304      0   5800    102      0
4        800.02      5     62     0      4  ...  263      0   5817    102      0
...         ...    ...    ...   ...    ...  ...  ...    ...    ...    ...    ...
973938  1154.25      5     62     0      5  ...  357      0   5817    102      0
973939   366.38      5     62     0      5  ...  291      0   2260    102      0
973940   352.55      5     62     0      5  ...  251      0   4907    102      0
973941   168.20      5     62     1      2  ...  380      0   6580     38      0
973942   834.05      5     62     1      8  ...  373      0   5817    102      0

[973943 rows x 17 columns]


### Training

In [ ]:
# specify your configurations as a dict

lgb_param = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'binary',
    'learning_rate' : 0.1,
}


In [ ]:
print('Starting training...')
max_round = 100
gbm = lgb.train(lgb_param,
         lgb_train,
         num_boost_round=max_round,
         valid_sets=lgb_eval,
         early_stopping_rounds=max_round)

Starting training...
[1]	valid_0's binary_logloss: 0.0454659
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.041861
[3]	valid_0's binary_logloss: 0.0401136
[4]	valid_0's binary_logloss: 0.038948
[5]	valid_0's binary_logloss: 0.0378365
[6]	valid_0's binary_logloss: 0.0369821
[7]	valid_0's binary_logloss: 0.0359441
[8]	valid_0's binary_logloss: 0.0352906
[9]	valid_0's binary_logloss: 0.0346922
[10]	valid_0's binary_logloss: 0.0341762
[11]	valid_0's binary_logloss: 0.0337284
[12]	valid_0's binary_logloss: 0.0333063
[13]	valid_0's binary_logloss: 0.0329253
[14]	valid_0's binary_logloss: 0.0325733
[15]	valid_0's binary_logloss: 0.0322681
[16]	valid_0's binary_logloss: 0.0320072
[17]	valid_0's binary_logloss: 0.0317156
[18]	valid_0's binary_logloss: 0.0315248
[19]	valid_0's binary_logloss: 0.0313137
[20]	valid_0's binary_logloss: 0.0311148
[21]	valid_0's binary_logloss: 0.0311914
[22]	valid_0's binary_logloss: 0.0310725
[23]	valid_0's binary_log

### Show the performance on testing set


In [ ]:
target_y = df_y_test
y_pred = gbm.predict(df_x_test, num_iteration=gbm.best_iteration)

# print(y_pred)
print('df_y_test.shape: {}'.format(df_x_test.shape))
print('auc: {}'.format(roc_auc_score(target_y, y_pred)))
th = 0.5

# row: actually, column: predict, the smaller class will be the target for counting the following indexies
cm = confusion_matrix(target_y, (y_pred > th).astype(int), labels=[0,1])
print(cm)
print('precision_score: {}'.format(precision_score(target_y, (y_pred > th).astype(int))))
print('recall_score: {}'.format(recall_score(target_y, (y_pred > th).astype(int))))
print('f1_score: {}'.format(f1_score(target_y, (y_pred > th).astype(int))))
print('accuracy: {}'.format(accuracy_score(target_y, (y_pred > th).astype(int))))



df_y_test.shape: (304358, 17)
auc: 0.9577935744364776
[[300970    559]
 [  2134    695]]
precision_score: 0.5542264752791068
recall_score: 0.24566984800282785
f1_score: 0.3404359539554249
accuracy: 0.9911518672090105


In [ ]:
# json_model = gbm.dump_model()
# import pprint
# pprint.pprint(json_model)

# Data Processing scenario 7
- Base on scenerio 3
- Do normalization on numerical features
- Only use the feature that selected by subjective
  - drop column : acquic, bacno, cano, mchno
- 64%(0.8 * 0.8) training data
- 16%(0.8 * 0.2) to validate
- 20% to test
- f1 : 0.34

In [ ]:
import pandas as pd
with open('/content/drive/My Drive/fintech-introduction/term-project/train_feature_normalization.csv', 'r') as f:
  df = pd.read_csv(f)
  # df = df.set_index('txkey')


# print(df_X)
# print(df_y)
df

,Unnamed: 0,acqic,bacno,cano,conam,mcc,mchno,scity,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd,hh,mm,ss,fraud_ind
0,0,0.505183,1.165242,1.106042,0.325808,-1.369912,-1.274220,0.536349,5,62,0,5,0,0,5,0,0,1,0,102,0,7,6,2,0
1,1,0.015972,0.275233,-0.306370,-0.816225,-0.600363,0.738111,0.536349,5,62,0,4,0,0,5,0,0,1,0,102,0,19,29,20,0
2,2,0.137776,0.483542,0.970861,-0.910290,1.054167,-0.053604,0.921739,5,62,1,2,0,0,5,0,0,1,0,38,0,16,1,24,0
3,3,0.471904,1.607659,-1.238400,0.352968,0.079405,-0.721353,0.527763,5,62,0,5,0,0,5,0,0,1,0,102,0,15,33,53,0
4,4,-0.021966,-0.070230,0.873330,0.360713,-0.446453,1.229661,0.536349,5,62,0,4,0,0,5,0,0,1,0,102,0,19,14,48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,1521782,0.406010,-0.345490,1.198779,-1.058028,-1.357086,0.310865,0.928305,5,62,1,2,0,0,5,0,0,6,0,75,0,16,54,27,0
1521783,1521783,0.506515,-0.820698,-1.482458,0.707579,0.964387,0.756247,0.536349,5,62,1,8,0,1,5,0,0,6,0,102,0,12,42,32,0
1521784,1521784,0.505183,-1.340286,-1.205217,-0.672683,-0.651666,-1.602903,0.536349,5,62,0,5,0,0,5,0,0,6,0,102,0,14,1,37,0
1521785,1521785,0.581061,-1.572495,-1.163167,-0.349849,2.041755,0.111695,-2.401805,5,0,0,0,0,0,5,0,0,6,0,102,0,18,2,43,0


In [ ]:
df_y = df['fraud_ind']
df_x = df.drop(['fraud_ind', 'Unnamed: 0', 'acqic', 'bacno', 'cano', 'mchno'], 1)
df_x

,conam,mcc,scity,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd,hh,mm,ss
0,0.325808,-1.369912,0.536349,5,62,0,5,0,0,5,0,0,1,0,102,0,7,6,2
1,-0.816225,-0.600363,0.536349,5,62,0,4,0,0,5,0,0,1,0,102,0,19,29,20
2,-0.910290,1.054167,0.921739,5,62,1,2,0,0,5,0,0,1,0,38,0,16,1,24
3,0.352968,0.079405,0.527763,5,62,0,5,0,0,5,0,0,1,0,102,0,15,33,53
4,0.360713,-0.446453,0.536349,5,62,0,4,0,0,5,0,0,1,0,102,0,19,14,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,-1.058028,-1.357086,0.928305,5,62,1,2,0,0,5,0,0,6,0,75,0,16,54,27
1521783,0.707579,0.964387,0.536349,5,62,1,8,0,1,5,0,0,6,0,102,0,12,42,32
1521784,-0.672683,-0.651666,0.536349,5,62,0,5,0,0,5,0,0,6,0,102,0,14,1,37
1521785,-0.349849,2.041755,-2.401805,5,0,0,0,0,0,5,0,0,6,0,102,0,18,2,43


In [ ]:
# split train+val and test
df_x_data, df_x_test = train_test_split(df_x, test_size=0.2, shuffle=False)
df_y_data = df_y.loc[df_x_data.index]
df_y_test = df_y.loc[df_x_test.index]

# print(df_x_data)
# print(df_x_test)
# print(df_y_data)
# print(df_y_test)

#split train and val
df_x_train, df_x_val = train_test_split(df_x_data, test_size=0.2, shuffle=False)
df_y_train = df_y_data.loc[df_x_train.index]
df_y_val = df_y_data.loc[df_x_val.index]


print('df_x_train:')
print(df_x_train)
print('df_x_val:')
print(df_x_val)
print('df_x_test:')
print(df_x_test)
print('df_y_train:')
print(df_y_train)
print('df_y_val:')
print(df_y_val)
print('df_y_test:')
print(df_y_test)

df_x_train:
           conam       mcc     scity  contp  csmcu  ...  stocn  stscd  hh  mm  ss
0       0.325808 -1.369912  0.536349      5     62  ...    102      0   7   6   2
1      -0.816225 -0.600363  0.536349      5     62  ...    102      0  19  29  20
2      -0.910290  1.054167  0.921739      5     62  ...     38      0  16   1  24
3       0.352968  0.079405  0.527763      5     62  ...    102      0  15  33  53
4       0.360713 -0.446453  0.536349      5     62  ...    102      0  19  14  48
...          ...       ...       ...    ...    ...  ...    ...    ...  ..  ..  ..
973938  1.240116  0.759174  0.536349      5     62  ...    102      0  19  30  19
973939 -0.715830 -0.087330 -1.260284      5     62  ...    102      0  16  36  56
973940 -0.750164 -0.600363  0.076710      5     62  ...    102      0  20  29   1
973941 -1.207827  1.054167  0.921739      5     62  ...     38      0  16  29  58
973942  0.445195  0.964387  0.536349      5     62  ...    102      0  18  20  33

[97

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_x_train, df_y_train)
lgb_eval = lgb.Dataset(df_x_val, df_y_val)
print(lgb_train.data)
print(lgb_eval.data)

           conam       mcc     scity  contp  csmcu  ...  stocn  stscd  hh  mm  ss
0       0.325808 -1.369912  0.536349      5     62  ...    102      0   7   6   2
1      -0.816225 -0.600363  0.536349      5     62  ...    102      0  19  29  20
2      -0.910290  1.054167  0.921739      5     62  ...     38      0  16   1  24
3       0.352968  0.079405  0.527763      5     62  ...    102      0  15  33  53
4       0.360713 -0.446453  0.536349      5     62  ...    102      0  19  14  48
...          ...       ...       ...    ...    ...  ...    ...    ...  ..  ..  ..
973938  1.240116  0.759174  0.536349      5     62  ...    102      0  19  30  19
973939 -0.715830 -0.087330 -1.260284      5     62  ...    102      0  16  36  56
973940 -0.750164 -0.600363  0.076710      5     62  ...    102      0  20  29   1
973941 -1.207827  1.054167  0.921739      5     62  ...     38      0  16  29  58
973942  0.445195  0.964387  0.536349      5     62  ...    102      0  18  20  33

[973943 rows x 

### Training


In [ ]:
# specify your configurations as a dict

lgb_param = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'binary_logloss',
    'learning_rate' : 0.1,
}


In [ ]:
print('Starting training...')
max_round = 100
gbm = lgb.train(lgb_param,
         lgb_train,
         num_boost_round=max_round,
         valid_sets=lgb_eval,
         early_stopping_rounds=max_round)

Starting training...
[1]	valid_0's binary_logloss: 0.0456438
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.0423656
[3]	valid_0's binary_logloss: 0.0408676
[4]	valid_0's binary_logloss: 0.0394893
[5]	valid_0's binary_logloss: 0.0384035
[6]	valid_0's binary_logloss: 0.0375022
[7]	valid_0's binary_logloss: 0.0367857
[8]	valid_0's binary_logloss: 0.0362772
[9]	valid_0's binary_logloss: 0.0356706
[10]	valid_0's binary_logloss: 0.0347511
[11]	valid_0's binary_logloss: 0.0342253
[12]	valid_0's binary_logloss: 0.0337749
[13]	valid_0's binary_logloss: 0.0333533
[14]	valid_0's binary_logloss: 0.0330031
[15]	valid_0's binary_logloss: 0.0326839
[16]	valid_0's binary_logloss: 0.0324164
[17]	valid_0's binary_logloss: 0.0321548
[18]	valid_0's binary_logloss: 0.0319253
[19]	valid_0's binary_logloss: 0.0317084
[20]	valid_0's binary_logloss: 0.0315474
[21]	valid_0's binary_logloss: 0.0313579
[22]	valid_0's binary_logloss: 0.0312945
[23]	valid_0's binary_l

### Show the performance on testing set

In [ ]:
target_y = df_y_test
y_pred = gbm.predict(df_x_test, num_iteration=gbm.best_iteration)

# print(y_pred)
print('df_y_test.shape: {}'.format(df_x_test.shape))
print('auc: {}'.format(roc_auc_score(target_y, y_pred)))
th = 0.5

# row: actually, column: predict, the smaller class will be the target for counting the following indexies
cm = confusion_matrix(target_y, (y_pred > th).astype(int), labels=[0,1])
print(cm)
print('precision_score: {}'.format(precision_score(target_y, (y_pred > th).astype(int))))
print('recall_score: {}'.format(recall_score(target_y, (y_pred > th).astype(int))))
print('f1_score: {}'.format(f1_score(target_y, (y_pred > th).astype(int))))
print('accuracy: {}'.format(accuracy_score(target_y, (y_pred > th).astype(int))))



df_y_test.shape: (304358, 19)
auc: 0.958600783560829
[[300833    696]
 [  2103    726]]
precision_score: 0.510548523206751
recall_score: 0.256627783669141
f1_score: 0.34156669019054337
accuracy: 0.9908035931370294


# Data Processing scenario 8
- Base on scenerio 3
- Do normalization on numerical features
- Only use the feature that selected by subjective
  - use column : hh, mm, ss, locdt, stocn, hecfg, ecfg
- 64%(0.8 * 0.8) training data
- 16%(0.8 * 0.2) to validate
- 20% to test
- f1 : 

In [ ]:
import pandas as pd
with open('/content/drive/My Drive/fintech-introduction/term-project/train_feature_normalization.csv', 'r') as f:
  df = pd.read_csv(f)
  # df = df.set_index('txkey')


# print(df_X)
# print(df_y)
df

,Unnamed: 0,acqic,bacno,cano,conam,mcc,mchno,scity,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd,hh,mm,ss,fraud_ind
0,0,0.505183,1.165242,1.106042,0.325808,-1.369912,-1.274220,0.536349,5,62,0,5,0,0,5,0,0,1,0,102,0,7,6,2,0
1,1,0.015972,0.275233,-0.306370,-0.816225,-0.600363,0.738111,0.536349,5,62,0,4,0,0,5,0,0,1,0,102,0,19,29,20,0
2,2,0.137776,0.483542,0.970861,-0.910290,1.054167,-0.053604,0.921739,5,62,1,2,0,0,5,0,0,1,0,38,0,16,1,24,0
3,3,0.471904,1.607659,-1.238400,0.352968,0.079405,-0.721353,0.527763,5,62,0,5,0,0,5,0,0,1,0,102,0,15,33,53,0
4,4,-0.021966,-0.070230,0.873330,0.360713,-0.446453,1.229661,0.536349,5,62,0,4,0,0,5,0,0,1,0,102,0,19,14,48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,1521782,0.406010,-0.345490,1.198779,-1.058028,-1.357086,0.310865,0.928305,5,62,1,2,0,0,5,0,0,6,0,75,0,16,54,27,0
1521783,1521783,0.506515,-0.820698,-1.482458,0.707579,0.964387,0.756247,0.536349,5,62,1,8,0,1,5,0,0,6,0,102,0,12,42,32,0
1521784,1521784,0.505183,-1.340286,-1.205217,-0.672683,-0.651666,-1.602903,0.536349,5,62,0,5,0,0,5,0,0,6,0,102,0,14,1,37,0
1521785,1521785,0.581061,-1.572495,-1.163167,-0.349849,2.041755,0.111695,-2.401805,5,0,0,0,0,0,5,0,0,6,0,102,0,18,2,43,0


In [ ]:
df_y = df['fraud_ind']
df_x = df[['hh']]
df_x

,hh
0,7
1,19
2,16
3,15
4,19
...,...
1521782,16
1521783,12
1521784,14
1521785,18


In [ ]:
# split train+val and test
df_x_data, df_x_test = train_test_split(df_x, test_size=0.2, shuffle=False)
df_y_data = df_y.loc[df_x_data.index]
df_y_test = df_y.loc[df_x_test.index]

# print(df_x_data)
# print(df_x_test)
# print(df_y_data)
# print(df_y_test)

#split train and val
df_x_train, df_x_val = train_test_split(df_x_data, test_size=0.2, shuffle=False)
df_y_train = df_y_data.loc[df_x_train.index]
df_y_val = df_y_data.loc[df_x_val.index]


print('df_x_train:')
print(df_x_train)
print('df_x_val:')
print(df_x_val)
print('df_x_test:')
print(df_x_test)
print('df_y_train:')
print(df_y_train)
print('df_y_val:')
print(df_y_val)
print('df_y_test:')
print(df_y_test)

df_x_train:
        hh
0        7
1       19
2       16
3       15
4       19
...     ..
973938  19
973939  16
973940  20
973941  16
973942  18

[973943 rows x 1 columns]
df_x_val:
         hh
973943   20
973944   20
973945   15
973946   13
973947   18
...      ..
1217424  19
1217425  15
1217426   9
1217427  16
1217428  13

[243486 rows x 1 columns]
df_x_test:
         hh
1217429   9
1217430  17
1217431  20
1217432  17
1217433  18
...      ..
1521782  16
1521783  12
1521784  14
1521785  18
1521786  15

[304358 rows x 1 columns]
df_y_train:
0         0
1         0
2         0
3         0
4         0
         ..
973938    0
973939    0
973940    0
973941    0
973942    0
Name: fraud_ind, Length: 973943, dtype: int64
df_y_val:
973943     0
973944     0
973945     0
973946     0
973947     0
          ..
1217424    0
1217425    0
1217426    0
1217427    0
1217428    0
Name: fraud_ind, Length: 243486, dtype: int64
df_y_test:
1217429    0
1217430    0
1217431    0
1217432    0
1217433    0
 

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_x_train, df_y_train)
lgb_eval = lgb.Dataset(df_x_val, df_y_val)
print(lgb_train.data)
print(lgb_eval.data)

        hh
0        7
1       19
2       16
3       15
4       19
...     ..
973938  19
973939  16
973940  20
973941  16
973942  18

[973943 rows x 1 columns]
         hh
973943   20
973944   20
973945   15
973946   13
973947   18
...      ..
1217424  19
1217425  15
1217426   9
1217427  16
1217428  13

[243486 rows x 1 columns]


### Training


In [ ]:
# specify your configurations as a dict

lgb_param = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'binary_logloss',
    'learning_rate' : 0.1,
}


In [ ]:
print('Starting training...')
max_round = 100
gbm = lgb.train(lgb_param,
         lgb_train,
         num_boost_round=max_round,
         valid_sets=lgb_eval,
         early_stopping_rounds=max_round)

Starting training...
[1]	valid_0's binary_logloss: 0.0613338
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.0608611
[3]	valid_0's binary_logloss: 0.0605501
[4]	valid_0's binary_logloss: 0.0603315
[5]	valid_0's binary_logloss: 0.0601728
[6]	valid_0's binary_logloss: 0.0600551
[7]	valid_0's binary_logloss: 0.059967
[8]	valid_0's binary_logloss: 0.0599005
[9]	valid_0's binary_logloss: 0.0598504
[10]	valid_0's binary_logloss: 0.0598125
[11]	valid_0's binary_logloss: 0.0597841
[12]	valid_0's binary_logloss: 0.0597629
[13]	valid_0's binary_logloss: 0.0597472
[14]	valid_0's binary_logloss: 0.0597359
[15]	valid_0's binary_logloss: 0.0597279
[16]	valid_0's binary_logloss: 0.0597224
[17]	valid_0's binary_logloss: 0.0597188
[18]	valid_0's binary_logloss: 0.0597167
[19]	valid_0's binary_logloss: 0.0597158
[20]	valid_0's binary_logloss: 0.0597156
[21]	valid_0's binary_logloss: 0.0597161
[22]	valid_0's binary_logloss: 0.0597169
[23]	valid_0's binary_lo

### Show the performance on testing set

In [ ]:
target_y = df_y_test
y_pred = gbm.predict(df_x_test, num_iteration=gbm.best_iteration)

# print(y_pred)
print('df_y_test.shape: {}'.format(df_x_test.shape))
print('auc: {}'.format(roc_auc_score(target_y, y_pred)))
th = 0.5

# row: actually, column: predict, the smaller class will be the target for counting the following indexies
cm = confusion_matrix(target_y, (y_pred > th).astype(int), labels=[0,1])
print(cm)
print('precision_score: {}'.format(precision_score(target_y, (y_pred > th).astype(int))))
print('recall_score: {}'.format(recall_score(target_y, (y_pred > th).astype(int))))
print('f1_score: {}'.format(f1_score(target_y, (y_pred > th).astype(int))))
print('accuracy: {}'.format(accuracy_score(target_y, (y_pred > th).astype(int))))



df_y_test.shape: (304358, 1)
auc: 0.6416224757565612
[[301529      0]
 [  2829      0]]
precision_score: 0.0
recall_score: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


f1_score: 0.0
accuracy: 0.9907050250034499


# Data Processing scenario 9
- Base on scenerio 3
- Do normalization on numerical features
- Only use the feature that selected by subjective
  - use column : hh, mm, ss, locdt, stocn, hcefg, ecfg
- balance two class
- 64%(0.8 * 0.8) training data
- 16%(0.8 * 0.2) to validate
- 20% to test
- f1 : 

In [ ]:
import pandas as pd
with open('/content/drive/My Drive/fintech-introduction/term-project/train_feature_normalization.csv', 'r') as f:
  df = pd.read_csv(f)
  # df = df.set_index('txkey')


# print(df_X)
# print(df_y)
df

,Unnamed: 0,acqic,bacno,cano,conam,mcc,mchno,scity,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd,hh,mm,ss,fraud_ind
0,0,0.505183,1.165242,1.106042,0.325808,-1.369912,-1.274220,0.536349,5,62,0,5,0,0,5,0,0,1,0,102,0,7,6,2,0
1,1,0.015972,0.275233,-0.306370,-0.816225,-0.600363,0.738111,0.536349,5,62,0,4,0,0,5,0,0,1,0,102,0,19,29,20,0
2,2,0.137776,0.483542,0.970861,-0.910290,1.054167,-0.053604,0.921739,5,62,1,2,0,0,5,0,0,1,0,38,0,16,1,24,0
3,3,0.471904,1.607659,-1.238400,0.352968,0.079405,-0.721353,0.527763,5,62,0,5,0,0,5,0,0,1,0,102,0,15,33,53,0
4,4,-0.021966,-0.070230,0.873330,0.360713,-0.446453,1.229661,0.536349,5,62,0,4,0,0,5,0,0,1,0,102,0,19,14,48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,1521782,0.406010,-0.345490,1.198779,-1.058028,-1.357086,0.310865,0.928305,5,62,1,2,0,0,5,0,0,6,0,75,0,16,54,27,0
1521783,1521783,0.506515,-0.820698,-1.482458,0.707579,0.964387,0.756247,0.536349,5,62,1,8,0,1,5,0,0,6,0,102,0,12,42,32,0
1521784,1521784,0.505183,-1.340286,-1.205217,-0.672683,-0.651666,-1.602903,0.536349,5,62,0,5,0,0,5,0,0,6,0,102,0,14,1,37,0
1521785,1521785,0.581061,-1.572495,-1.163167,-0.349849,2.041755,0.111695,-2.401805,5,0,0,0,0,0,5,0,0,6,0,102,0,18,2,43,0


In [ ]:
df_y = df['fraud_ind']
df_x = df[[ 'hh', 'locdt', 'stocn', 'hcefg', 'ecfg']]
df_x

,hh,locdt,stocn,hcefg,ecfg
0,7,1,102,5,0
1,19,1,102,5,0
2,16,1,38,5,1
3,15,1,102,5,0
4,19,1,102,5,0
...,...,...,...,...,...
1521782,16,6,75,5,1
1521783,12,6,102,5,1
1521784,14,6,102,5,0
1521785,18,6,102,5,0


In [ ]:
# split train+val and test
df_x_data, df_x_test = train_test_split(df_x, test_size=0.2, shuffle=False)
df_y_data = df_y.loc[df_x_data.index]
df_y_test = df_y.loc[df_x_test.index]

# print(df_x_data)
# print(df_x_test)
# print(df_y_data)
# print(df_y_test)

#split train and val
df_x_train, df_x_val = train_test_split(df_x_data, test_size=0.2, shuffle=False)
df_y_train = df_y_data.loc[df_x_train.index]
df_y_val = df_y_data.loc[df_x_val.index]


print('df_x_train:')
print(df_x_train)
print('df_x_val:')
print(df_x_val)
print('df_x_test:')
print(df_x_test)
print('df_y_train:')
print(df_y_train)
print('df_y_val:')
print(df_y_val)
print('df_y_test:')
print(df_y_test)

df_x_train:
        hh  locdt  stocn  hcefg  ecfg
0        7      1    102      5     0
1       19      1    102      5     0
2       16      1     38      5     1
3       15      1    102      5     0
4       19      1    102      5     0
...     ..    ...    ...    ...   ...
973938  19      2    102      5     0
973939  16      2    102      1     0
973940  20      2    102      8     0
973941  16      2     38      5     1
973942  18      2    102      5     1

[973943 rows x 5 columns]
df_x_val:
         hh  locdt  stocn  hcefg  ecfg
973943   20      2    102      5     0
973944   20      2    102      5     0
973945   15      2    102      5     0
973946   13      2    102      5     0
973947   18      2    102      5     0
...      ..    ...    ...    ...   ...
1217424  19      2    102      5     0
1217425  15      2    102      5     0
1217426   9      2    102      5     0
1217427  16      2    102      5     0
1217428  13      2    102      5     0

[243486 rows x 5 columns]


In [ ]:
# Balance two #example in two class
# df_y_train
df_fraud = df_x_train.loc[df_y_train == 1]
df_non_fraud = df_x_train.loc[df_y_train == 0].sample(n=df_fraud.shape[0])
df_x_train_balance = pd.concat([df_fraud, df_non_fraud])
df_y_train_balance = pd.concat([df_y_train.loc[df_fraud.index], df_y_train.loc[df_non_fraud.index]])
df_fraud, df_non_fraud, df_x_train_balance, df_y_train_balance


(        hh  mm  locdt  stocn  hcefg  ecfg
 60      17  13      1     38      5     0
 157     15  21      1     75      5     0
 208     17  23      1     27      5     1
 224     21  58      1     46      5     1
 274     21  28      1     52      5     1
 ...     ..  ..    ...    ...    ...   ...
 973222  21   4      2     38      5     1
 973509  13  39      2     46      5     1
 973717  14  34      2     38      5     1
 973843   5  28      2     75      5     1
 973924   2  31      2     75      5     1
 
 [14789 rows x 6 columns],         hh  mm  locdt  stocn  hcefg  ecfg
 88637   14   6      6    102      5     1
 37857   11   7      3     20      5     1
 197606  22   4      5     20      5     1
 228032  13  46      0    102      5     0
 68960   14  37      4    102      5     0
 ...     ..  ..    ...    ...    ...   ...
 847807  14  13      2    102      5     1
 179510  16  35      4    102      5     0
 213370  16  56      6     20      5     1
 78645   19  32      5    

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_x_train, df_y_train)
lgb_eval = lgb.Dataset(df_x_val, df_y_val)
print(lgb_train.data)
print(lgb_eval.data)

        hh  locdt  stocn  hcefg  ecfg
0        7      1    102      5     0
1       19      1    102      5     0
2       16      1     38      5     1
3       15      1    102      5     0
4       19      1    102      5     0
...     ..    ...    ...    ...   ...
973938  19      2    102      5     0
973939  16      2    102      1     0
973940  20      2    102      8     0
973941  16      2     38      5     1
973942  18      2    102      5     1

[973943 rows x 5 columns]
         hh  locdt  stocn  hcefg  ecfg
973943   20      2    102      5     0
973944   20      2    102      5     0
973945   15      2    102      5     0
973946   13      2    102      5     0
973947   18      2    102      5     0
...      ..    ...    ...    ...   ...
1217424  19      2    102      5     0
1217425  15      2    102      5     0
1217426   9      2    102      5     0
1217427  16      2    102      5     0
1217428  13      2    102      5     0

[243486 rows x 5 columns]


### Training


In [ ]:
# specify your configurations as a dict

lgb_param = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'binary_logloss',
    'learning_rate' : 0.1,
}


In [ ]:
print('Starting training...')
max_round = 100
gbm = lgb.train(lgb_param,
         lgb_train,
         num_boost_round=max_round,
         valid_sets=lgb_eval,
         early_stopping_rounds=max_round)

Starting training...
[1]	valid_0's binary_logloss: 0.0526702
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.050082
[3]	valid_0's binary_logloss: 0.0482456
[4]	valid_0's binary_logloss: 0.0468356
[5]	valid_0's binary_logloss: 0.0457011
[6]	valid_0's binary_logloss: 0.0447793
[7]	valid_0's binary_logloss: 0.0439891
[8]	valid_0's binary_logloss: 0.0433242
[9]	valid_0's binary_logloss: 0.0427588
[10]	valid_0's binary_logloss: 0.0422642
[11]	valid_0's binary_logloss: 0.041824
[12]	valid_0's binary_logloss: 0.0414586
[13]	valid_0's binary_logloss: 0.0411313
[14]	valid_0's binary_logloss: 0.0408416
[15]	valid_0's binary_logloss: 0.0405998
[16]	valid_0's binary_logloss: 0.0403703
[17]	valid_0's binary_logloss: 0.0401695
[18]	valid_0's binary_logloss: 0.0399995
[19]	valid_0's binary_logloss: 0.0398566
[20]	valid_0's binary_logloss: 0.039736
[21]	valid_0's binary_logloss: 0.0396157
[22]	valid_0's binary_logloss: 0.0395192
[23]	valid_0's binary_logl

### Show the performance on testing set

In [ ]:
target_y = df_y_test
y_pred = gbm.predict(df_x_test, num_iteration=gbm.best_iteration)

# print(y_pred)
print('df_y_test.shape: {}'.format(df_x_test.shape))
print('auc: {}'.format(roc_auc_score(target_y, y_pred)))
th = 0.5

# row: actually, column: predict, the smaller class will be the target for counting the following indexies
cm = confusion_matrix(target_y, (y_pred > th).astype(int), labels=[0,1])
print(cm)
print('precision_score: {}'.format(precision_score(target_y, (y_pred > th).astype(int))))
print('recall_score: {}'.format(recall_score(target_y, (y_pred > th).astype(int))))
print('f1_score: {}'.format(f1_score(target_y, (y_pred > th).astype(int))))
print('accuracy: {}'.format(accuracy_score(target_y, (y_pred > th).astype(int))))



df_y_test.shape: (304358, 5)
auc: 0.9419269768383172
[[301449     80]
 [  2800     29]]
precision_score: 0.26605504587155965
recall_score: 0.010250972074938141
f1_score: 0.01974132062627638
accuracy: 0.9905374591763647


# Data Processing scenario 10
- Base on scenerio 3
- Only use the feature that selected by subjective
  - use column : hh, mm, ss, locdt, stocn, hcefg, ecfg
- Do normalization on numerical features
- Convert catogorical featrues to one hot style
- Only about 400000 data in total
- 64%(0.8 * 0.8) training data
- 16%(0.8 * 0.2) to validate
- 20% to test
- f1 : 0.33

In [ ]:
import pandas as pd
with open('/content/drive/My Drive/fintech-introduction/term-project/train_feature_normalization_onehot_subjective.csv', 'r') as f:
  df = pd.read_csv(f)
  # df = df.set_index('txkey')


# print(df_X)
# print(df_y)
df

,Unnamed: 0,acqic,bacno,cano,conam,mcc,mchno,scity,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,hh_8,hh_9,hh_10,hh_11,hh_12,hh_13,hh_14,hh_15,hh_16,hh_17,hh_18,hh_19,hh_20,hh_21,hh_22,hh_23,locdt_0,locdt_1,locdt_2,locdt_3,locdt_4,locdt_5,locdt_6,stocn_0,...,stocn_79,stocn_80,stocn_81,stocn_82,stocn_83,stocn_84,stocn_85,stocn_86,stocn_87,stocn_88,stocn_89,stocn_90,stocn_91,stocn_92,stocn_93,stocn_94,stocn_95,stocn_96,stocn_97,stocn_98,stocn_99,stocn_100,stocn_101,stocn_102,stocn_104,stocn_105,stocn_106,stocn_107,hcefg_0,hcefg_1,hcefg_2,hcefg_3,hcefg_5,hcefg_6,hcefg_7,hcefg_8,hcefg_9,ecfg_0,ecfg_1,fraud_ind
0,3,0.471904,1.607659,-1.238400,0.352968,0.079405,-0.721353,0.527763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
1,6,0.506515,1.151497,1.254999,-0.463005,-0.074504,0.753360,0.536349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,11,0.505183,-1.525137,-1.766940,-1.402485,-0.613189,-1.160214,0.536349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,12,0.148425,0.513185,1.219648,-0.628915,-1.369912,-1.150319,0.536349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,13,0.214985,0.787221,1.045652,-0.838916,0.797651,-1.422226,-0.114217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760889,1521781,-1.874977,-0.194189,1.676025,0.524985,0.258967,0.412445,0.594436,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
760890,1521782,0.406010,-0.345490,1.198779,-1.058028,-1.357086,0.310865,0.928305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
760891,1521783,0.506515,-0.820698,-1.482458,0.707579,0.964387,0.756247,0.536349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
760892,1521784,0.505183,-1.340286,-1.205217,-0.672683,-0.651666,-1.602903,0.536349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0

In [ ]:
df_y = df['fraud_ind']
df_x = df.drop(['fraud_ind', 'Unnamed: 0'], 1)
df_x

,acqic,bacno,cano,conam,mcc,mchno,scity,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,hh_8,hh_9,hh_10,hh_11,hh_12,hh_13,hh_14,hh_15,hh_16,hh_17,hh_18,hh_19,hh_20,hh_21,hh_22,hh_23,locdt_0,locdt_1,locdt_2,locdt_3,locdt_4,locdt_5,locdt_6,stocn_0,stocn_1,...,stocn_78,stocn_79,stocn_80,stocn_81,stocn_82,stocn_83,stocn_84,stocn_85,stocn_86,stocn_87,stocn_88,stocn_89,stocn_90,stocn_91,stocn_92,stocn_93,stocn_94,stocn_95,stocn_96,stocn_97,stocn_98,stocn_99,stocn_100,stocn_101,stocn_102,stocn_104,stocn_105,stocn_106,stocn_107,hcefg_0,hcefg_1,hcefg_2,hcefg_3,hcefg_5,hcefg_6,hcefg_7,hcefg_8,hcefg_9,ecfg_0,ecfg_1
0,0.471904,1.607659,-1.238400,0.352968,0.079405,-0.721353,0.527763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.506515,1.151497,1.254999,-0.463005,-0.074504,0.753360,0.536349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.505183,-1.525137,-1.766940,-1.402485,-0.613189,-1.160214,0.536349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.148425,0.513185,1.219648,-0.628915,-1.369912,-1.150319,0.536349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.214985,0.787221,1.045652,-0.838916,0.797651,-1.422226,-0.114217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760889,-1.874977,-0.194189,1.676025,0.524985,0.258967,0.412445,0.594436,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
760890,0.406010,-0.345490,1.198779,-1.058028,-1.357086,0.310865,0.928305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
760891,0.506515,-0.820698,-1.482458,0.707579,0.964387,0.756247,0.536349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
760892,0.505183,-1.340286,-1.205217,-0.672683,-0.651666,-1.602903,0.536349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,

In [ ]:
# split train+val and test
df_x_data, df_x_test = train_test_split(df_x, test_size=0.2, shuffle=False)
df_y_data = df_y.loc[df_x_data.index]
df_y_test = df_y.loc[df_x_test.index]

# print(df_x_data)
# print(df_x_test)
# print(df_y_data)
# print(df_y_test)

#split train and val
df_x_train, df_x_val = train_test_split(df_x_data, test_size=0.2, shuffle=False)
df_y_train = df_y_data.loc[df_x_train.index]
df_y_val = df_y_data.loc[df_x_val.index]


print('df_x_train:')
print(df_x_train)
print('df_x_val:')
print(df_x_val)
print('df_x_test:')
print(df_x_test)
print('df_y_train:')
print(df_y_train)
print('df_y_val:')
print(df_y_val)
print('df_y_test:')
print(df_y_test)

df_x_train:
           acqic     bacno      cano  ...  hcefg_9  ecfg_0  ecfg_1
0       0.471904  1.607659 -1.238400  ...      0.0     1.0     0.0
1       0.506515  1.151497  1.254999  ...      0.0     1.0     0.0
2       0.505183 -1.525137 -1.766940  ...      0.0     0.0     1.0
3       0.148425  0.513185  1.219648  ...      0.0     0.0     1.0
4       0.214985  0.787221  1.045652  ...      0.0     0.0     1.0
...          ...       ...       ...  ...      ...     ...     ...
486967  0.506515  1.167374  1.132921  ...      0.0     1.0     0.0
486968  0.120471 -0.946916  0.823350  ...      0.0     1.0     0.0
486969  0.129789  0.623822  0.007651  ...      0.0     0.0     1.0
486970  0.581061 -0.277504 -1.782260  ...      0.0     1.0     0.0
486971  0.381383  1.251048  1.336291  ...      0.0     1.0     0.0

[486972 rows x 152 columns]
df_x_val:
           acqic     bacno      cano  ...  hcefg_9  ecfg_0  ecfg_1
486972  0.506515  1.344265  0.312132  ...      0.0     1.0     0.0
486973  0.4

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_x_train, df_y_train)
lgb_eval = lgb.Dataset(df_x_val, df_y_val)
print(lgb_train.data)
print(lgb_eval.data)

           acqic     bacno      cano  ...  hcefg_9  ecfg_0  ecfg_1
0       0.471904  1.607659 -1.238400  ...      0.0     1.0     0.0
1       0.506515  1.151497  1.254999  ...      0.0     1.0     0.0
2       0.505183 -1.525137 -1.766940  ...      0.0     0.0     1.0
3       0.148425  0.513185  1.219648  ...      0.0     0.0     1.0
4       0.214985  0.787221  1.045652  ...      0.0     0.0     1.0
...          ...       ...       ...  ...      ...     ...     ...
486967  0.506515  1.167374  1.132921  ...      0.0     1.0     0.0
486968  0.120471 -0.946916  0.823350  ...      0.0     1.0     0.0
486969  0.129789  0.623822  0.007651  ...      0.0     0.0     1.0
486970  0.581061 -0.277504 -1.782260  ...      0.0     1.0     0.0
486971  0.381383  1.251048  1.336291  ...      0.0     1.0     0.0

[486972 rows x 152 columns]
           acqic     bacno      cano  ...  hcefg_9  ecfg_0  ecfg_1
486972  0.506515  1.344265  0.312132  ...      0.0     1.0     0.0
486973  0.471238  1.124999  0.444

### Training

In [ ]:
# specify your configurations as a dict

lgb_param = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'binary_logloss',
    'learning_rate' : 0.01,
}


In [ ]:
print('Starting training...')
max_round = 1000
gbm = lgb.train(lgb_param,
         lgb_train,
         num_boost_round=max_round,
         valid_sets=lgb_eval,
         early_stopping_rounds=max_round)

Starting training...
[1]	valid_0's binary_logloss: 0.0606917
Training until validation scores don't improve for 1000 rounds.
[2]	valid_0's binary_logloss: 0.0593092
[3]	valid_0's binary_logloss: 0.05814
[4]	valid_0's binary_logloss: 0.0571365
[5]	valid_0's binary_logloss: 0.0562685
[6]	valid_0's binary_logloss: 0.055473
[7]	valid_0's binary_logloss: 0.0547782
[8]	valid_0's binary_logloss: 0.0541205
[9]	valid_0's binary_logloss: 0.0535172
[10]	valid_0's binary_logloss: 0.0529478
[11]	valid_0's binary_logloss: 0.0524119
[12]	valid_0's binary_logloss: 0.0519443
[13]	valid_0's binary_logloss: 0.051472
[14]	valid_0's binary_logloss: 0.0510444
[15]	valid_0's binary_logloss: 0.050609
[16]	valid_0's binary_logloss: 0.0502054
[17]	valid_0's binary_logloss: 0.0498155
[18]	valid_0's binary_logloss: 0.0494604
[19]	valid_0's binary_logloss: 0.04911
[20]	valid_0's binary_logloss: 0.0487876
[21]	valid_0's binary_logloss: 0.0484524
[22]	valid_0's binary_logloss: 0.0481266
[23]	valid_0's binary_logloss

### Show the performance on testing set

In [ ]:
target_y = df_y_test
y_pred = gbm.predict(df_x_test, num_iteration=gbm.best_iteration)

# print(y_pred)
print('df_y_test.shape: {}'.format(df_x_test.shape))
print('auc: {}'.format(roc_auc_score(target_y, y_pred)))
th = 0.5

# row: actually, column: predict, the smaller class will be the target for counting the following indexies
cm = confusion_matrix(target_y, (y_pred > th).astype(int), labels=[0,1])
print(cm)
print('precision_score: {}'.format(precision_score(target_y, (y_pred > th).astype(int))))
print('recall_score: {}'.format(recall_score(target_y, (y_pred > th).astype(int))))
print('f1_score: {}'.format(f1_score(target_y, (y_pred > th).astype(int))))
print('accuracy: {}'.format(accuracy_score(target_y, (y_pred > th).astype(int))))



df_y_test.shape: (152179, 152)
auc: 0.9587848111243465
[[150532    252]
 [  1068    327]]
precision_score: 0.5647668393782384
recall_score: 0.23440860215053763
f1_score: 0.33130699088145893
accuracy: 0.991326004245001
